In [2]:
import numpy as np
import json
import scipy
from scipy import linalg, optimize,special
from recompilation_functions import random_gaussian_interactions
from qiskit import QuantumCircuit,assemble, Aer


In [3]:
#variables

omegas=[0.28,0.38,0.63,0.86] #qubits frequency

nr=100 #number of disorder realizations
nb_qubits=4 #system size
T=[2.5,5,10] #time to which the computation is performed

initial_state='ground_state'

backend = 'ibm_hanoi'
sessions=['thermalization 1','thermalization 2','thermalization 3']

nb_twirls_RC=100 #number of RC circuits per disorder realization

nb_shots_RC=1000 #number of shots per circuit

In [3]:
def IBU(ymes,t0,Rin,n):
    #This is the iterative Bayesian unfolding method.
    #Rin is a matrix where the first coordinate is the measured value and the second coordinate is the true value.
    #n is the number of iterations.
    
    tn = t0
    for q in range(n):
        out = []
        for j in range(len(t0)):
            mynum = 0.
            for i in range(len(ymes)):
                myden = 0.
                for k in range(len(t0)):
                    myden+=Rin[i][k]*tn[k]
                    pass
                mynum+=Rin[i][j]*tn[j]*ymes[i]/myden
                pass
            out+=[mynum]
        tn = out
        pass
    '''
    tn = t0
    for i in range(n):
        Rjitni = [np.array(Rin[:][i])*tn[i] for i in range(len(tn))]
        Pm_given_t = Rjitni / np.matmul(Rin,tn)
        tn = np.dot(Pm_given_t,ymes)
        pass
    '''
    return tn




In [4]:
#initialisation of the state for continuous time evolution
ini_state=[1/2**(nb_qubits/2)*(-1j)**sum([int(i) for i in bin(k)[2::]]) for k in range(2**nb_qubits)]
#measure in the y direction
y0=np.array([[1,-1j],[-1j,1]])#rotation  of an angle pi/2 along the x direction
meas_y=y0
for i in range(nb_qubits-1):
    meas_y=np.kron(meas_y,y0)
meas_y*=1/2**(nb_qubits/2)
nb_shots_sim=100000

t_array=np.arange(0,15,0.01)

pop_exact=list(list(np.zeros(2**nb_qubits) for _ in range(len(t_array)))for _ in range(nr))
pop_noiseless=list(list(list(0. for _ in range(nr)) for _ in range(2**nb_qubits)) for _ in range(len(T)))

for d in range(1,nr+1):
    
    J=np.load('data/circuits/disorder_realization_'+str(d)+'/random_interaction_'+str(d)+'.npy')
    print(d)
    print(J)
    for s,t in enumerate(T):
        qc=QuantumCircuit.from_qasm_file('data/circuits/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/recompiled_circuit_'+initial_state+'_disorder_'+str(d)+'_T='+str(round(t,1))+'.qasm')
        qobj = assemble(qc)
        sim_perfect = Aer.get_backend('aer_simulator')
        counts_perfect= sim_perfect.run(qobj,shots=nb_shots_sim).result().get_counts()
        for key in counts_perfect.keys():
            pop_noiseless[s][2**nb_qubits-int(key,2)-1][d-1]+=counts_perfect[key]/nb_shots_sim
    
    #simulation of the exact dynamics
    H=random_gaussian_interactions(J,omegas) #construct the hamiltonian
    vals,vecs=linalg.eigh(H) #find the eigenvalues and eigenvectors
    ini_ev=np.matmul(linalg.inv(vecs),ini_state) #write the initial state in the eigenbasis
    for k,t in enumerate(t_array):
        state=np.zeros_like(ini_ev)
        for i in range(2**nb_qubits):
            state[i]=np.exp(-1j*vals[i]*t)*ini_ev[i]#evolution of the state in the eigenbasis

        ev_state=np.linalg.multi_dot([meas_y,vecs,state]) #transform the state back to the original basis

        pop_exact[d-1][k]=np.abs(np.flip(ev_state))**2
    

1
[[ 0.00000000e+00+0.j          5.49415972e-03+0.00283998j
   2.04755901e-01+0.32497106j -6.67014334e-02-0.31669608j]
 [ 5.49415972e-03-0.00283998j  0.00000000e+00+0.j
  -3.47428316e-04-0.08453369j -6.96387186e-01-0.47773661j]
 [ 2.04755901e-01-0.32497106j -3.47428316e-04+0.08453369j
   0.00000000e+00+0.j          6.56540934e-02+0.44650085j]
 [-6.67014334e-02+0.31669608j -6.96387186e-01+0.47773661j
   6.56540934e-02-0.44650085j  0.00000000e+00+0.j        ]]
2
[[ 0.        +0.j          0.29394721-0.03801188j  0.3014511 -0.09167942j
   0.40720389+0.22348774j]
 [ 0.29394721+0.03801188j  0.        +0.j          0.45018303+0.52537117j
   0.18844697+0.09243614j]
 [ 0.3014511 +0.09167942j  0.45018303-0.52537117j  0.        +0.j
  -0.95436649+0.00957748j]
 [ 0.40720389-0.22348774j  0.18844697-0.09243614j -0.95436649-0.00957748j
   0.        +0.j        ]]
3
[[ 0.        +0.j         -0.69606486-0.02042105j -0.420096  -0.18502542j
   0.47162356+0.203702j  ]
 [-0.69606486+0.02042105j  0.      

22
[[ 0.        +0.j          0.19265282-0.46365373j  0.54797763-0.22567188j
  -0.29746564-0.03080902j]
 [ 0.19265282+0.46365373j  0.        +0.j         -0.4611767 -0.08268526j
  -0.72402975+0.21063055j]
 [ 0.54797763+0.22567188j -0.4611767 +0.08268526j  0.        +0.j
   0.05888014-0.23003535j]
 [-0.29746564+0.03080902j -0.72402975-0.21063055j  0.05888014+0.23003535j
   0.        +0.j        ]]
23
[[ 0.        +0.j          0.43073637+0.13072983j  0.21255844-0.04918367j
   0.18046861-0.06200922j]
 [ 0.43073637-0.13072983j  0.        +0.j          0.04944548-0.10306993j
  -0.56167675-0.29780919j]
 [ 0.21255844+0.04918367j  0.04944548+0.10306993j  0.        +0.j
  -0.58924723+0.18832389j]
 [ 0.18046861+0.06200922j -0.56167675+0.29780919j -0.58924723-0.18832389j
   0.        +0.j        ]]
24
[[ 0.        +0.j         -0.2186207 +0.2411698j  -0.07449061+0.33838408j
  -0.84242322-0.33350015j]
 [-0.2186207 -0.2411698j   0.        +0.j         -0.40870902+0.45565543j
   0.66233105-0.383331

43
[[ 0.        +0.j         -0.08639712+0.01986627j -0.14172326+0.35274239j
   0.01156403+0.0850205j ]
 [-0.08639712-0.01986627j  0.        +0.j          0.24696151+0.05748045j
   0.33038979+0.54616025j]
 [-0.14172326-0.35274239j  0.24696151-0.05748045j  0.        +0.j
  -0.33201671+0.33268627j]
 [ 0.01156403-0.0850205j   0.33038979-0.54616025j -0.33201671-0.33268627j
   0.        +0.j        ]]
44
[[ 0.        +0.j          0.17049284+0.59311926j -0.41968068-0.09378153j
  -0.64095095+0.0574937j ]
 [ 0.17049284-0.59311926j  0.        +0.j          0.2739857 +0.29648693j
   0.18341359+0.12805063j]
 [-0.41968068+0.09378153j  0.2739857 -0.29648693j  0.        +0.j
   0.11566837+0.06715651j]
 [-0.64095095-0.0574937j   0.18341359-0.12805063j  0.11566837-0.06715651j
   0.        +0.j        ]]
45
[[ 0.        +0.j         -0.12960068-0.53095084j -0.30789547+0.557735j
  -0.83520392-0.05574246j]
 [-0.12960068+0.53095084j  0.        +0.j         -0.65791335-0.42667315j
   0.12419703+0.06379982

64
[[ 0.        +0.j          0.21177825+0.18224714j -0.14206999-0.00227386j
   0.07953115-0.37656907j]
 [ 0.21177825-0.18224714j  0.        +0.j          0.10862782+0.28230741j
  -0.14101136+0.12253612j]
 [-0.14206999+0.00227386j  0.10862782-0.28230741j  0.        +0.j
   0.63635477+0.3575579j ]
 [ 0.07953115+0.37656907j -0.14101136-0.12253612j  0.63635477-0.3575579j
   0.        +0.j        ]]
65
[[ 0.        +0.j         -0.20062616+0.68558309j  0.14560922+0.42374305j
  -0.13665076-0.08684367j]
 [-0.20062616-0.68558309j  0.        +0.j         -0.31851348+0.09362433j
   0.15484653-0.03142841j]
 [ 0.14560922-0.42374305j -0.31851348-0.09362433j  0.        +0.j
  -0.68112239+0.52418146j]
 [-0.13665076+0.08684367j  0.15484653+0.03142841j -0.68112239-0.52418146j
   0.        +0.j        ]]
66
[[ 0.        +0.j          0.20292079-0.29742743j -0.68260187-0.35716778j
  -0.11096065+0.25875329j]
 [ 0.20292079+0.29742743j  0.        +0.j          0.03241387+0.558734j
   0.86468598+0.03242046j

85
[[ 0.        +0.j         -0.26147732+0.49772045j  0.0256428 -0.16339139j
  -0.23480333+0.46300933j]
 [-0.26147732-0.49772045j  0.        +0.j          0.3508704 -0.09406764j
  -0.11644718+0.30131656j]
 [ 0.0256428 +0.16339139j  0.3508704 +0.09406764j  0.        +0.j
   0.41210205-0.11394575j]
 [-0.23480333-0.46300933j -0.11644718-0.30131656j  0.41210205+0.11394575j
   0.        +0.j        ]]
86
[[ 0.        +0.j         -0.20628343-0.42126423j -0.03283242-0.07352338j
   0.76306393+0.12255982j]
 [-0.20628343+0.42126423j  0.        +0.j         -0.42427169-0.05937644j
   0.50032378+0.13817093j]
 [-0.03283242+0.07352338j -0.42427169+0.05937644j  0.        +0.j
  -0.14266266-0.38486927j]
 [ 0.76306393-0.12255982j  0.50032378-0.13817093j -0.14266266+0.38486927j
   0.        +0.j        ]]
87
[[ 0.        +0.j          0.36444893-0.01920026j  0.25252559-0.16254401j
  -0.27375193-0.0862527j ]
 [ 0.36444893+0.01920026j  0.        +0.j         -0.10016878-0.24443807j
  -0.39050956-0.190570

In [ ]:
pop_RC=list(list(list(list(0. for _ in range(nb_twirls_RC)) for _ in range(nr)) for _ in range(2**nb_qubits)) for _ in range(len(sessions)))
pop_ZNE3=list(list(list(list(0. for _ in range(nb_twirls_RC)) for _ in range(nr)) for _ in range(2**nb_qubits)) for _ in range(len(sessions)))
    
for s,session in enumerate(sessions):
    with open('data/quantum data/session '+str(session)+'/measurement_backend='+str(backend)+'.json',"r") as json_file:
        meas_matrix=json.load(json_file)


    print(meas_matrix)


    for d in range(1,nr+1):
        with open('data/quantum data/session '+str(session)+'/result_disorder_realization_'+str(d)+'_backend='+str(backend)+'.json',"r") as json_file:
            population=json.load(json_file)
        print(d)
        for k,pop in enumerate(population):
            counts_list=list(10**(-10)  for _ in range(2**nb_qubits))
            for key in pop.keys():
                counts_list[int(key)]=pop[key]
            counts_mitig=IBU(counts_list,np.ones(2**nb_qubits),meas_matrix,10)
            for b,c in enumerate(counts_mitig):
                pop_RC[s][2**nb_qubits-b-1][d-1][k]=c
        
        with open('data/quantum data/session '+str(session)+'/result_ZNE3_disorder_realization_'+str(d)+'_backend='+str(backend)+'.json',"r") as json_file:
            population=json.load(json_file)
        print(d)
        for k,pop in enumerate(population):
            counts_list=list(10**(-10)  for _ in range(2**nb_qubits))
            for key in pop.keys():
                counts_list[int(key)]=pop[key]
            counts_mitig=IBU(counts_list,np.ones(2**nb_qubits),meas_matrix,10)
            for b,c in enumerate(counts_mitig):
                pop_ZNE3[s][2**nb_qubits-b-1][d-1][k]=c

In [ ]:
np.save('data/populations/population_exact_dynamics_{}.npy'.format(initial_state),pop_exact)
np.save('data/populations/population_noiseless_{}.npy'.format(initial_state),pop_noiseless)
np.save('data/populations/population_RC_{}.npy'.format(initial_state),pop_RC)
np.save('data/populations/population_ZNE3_{}.npy'.format(initial_state),pop_ZNE3)

In [1]:
import numpy as np 
len(np.arange(0,15,0.01))

1500